## 1.Adım ) Test - Train olarak verileri ayırmak.Amacımız overfitting önlemek amacıyla Val.txt'leri kullanmak.Diğer tarafdan da .txt dosyalarının farklı fotoğraflardan oluşmadığını kontrol etmek için bu aşamayı yapıyoruz. (check_splits.py)

In [2]:
from pathlib import Path

def read_ids(txt_path: Path) -> set[str]:
    ids = set()
    if not txt_path.exists():
        raise FileNotFoundError(f"Bulunamadı: {txt_path}")
    for line in txt_path.read_text(encoding="utf-8", errors="ignore").splitlines():
        s = line.strip()
        if not s:
            continue
        s = s.replace(".jpg", "").replace(".png", "").replace(".jpeg", "")
        ids.add(s)
    return ids

def report_intersection(a_name: str, a: set[str], b_name: str, b: set[str], limit=30):
    inter = sorted(a & b)
    print(f"{a_name} ∩ {b_name}: {len(inter)}")
    if inter:
        print("  Örnek çakışan ID'ler:", inter[:limit])

def check_one_split(split_dir: Path, tag: str):
    print(f" {tag} ")
    train = read_ids(split_dir / "train.txt")
    val   = read_ids(split_dir / "val.txt")
    test  = read_ids(split_dir / "test.txt")

    print(f"train: {len(train)} | val: {len(val)} | test: {len(test)}")

    report_intersection("train", train, "val", val)
    report_intersection("train", train, "test", test)
    report_intersection("val", val, "test", test)

    union = train | val | test
    print(f"union(train,val,test): {len(union)}")

if __name__ == "__main__":
    PART_A_SPLIT_DIR = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Datasets\SCUT_HEAD_Part_A\ImageSets\Main")
    PART_B_SPLIT_DIR = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Datasets\SCUT_HEAD_Part_B\ImageSets\Main")

    check_one_split(PART_A_SPLIT_DIR, "Part A")
    check_one_split(PART_B_SPLIT_DIR, "Part B")

 Part A 
train: 1100 | val: 400 | test: 500
train ∩ val: 0
train ∩ test: 0
val ∩ test: 0
union(train,val,test): 2000
 Part B 
train: 1443 | val: 462 | test: 500
train ∩ val: 0
train ∩ test: 0
val ∩ test: 0
union(train,val,test): 2405


### Veri Seti Bölünmelerinin Doğrulanması

Eğitim sürecine başlamadan önce, veri sızıntısını (data leakage) önlemek amacıyla veri setindeki **eğitim (train)**, **doğrulama (validation)** ve **test** bölünmeleri kontrol edilmiştir.  
SCUT-HEAD veri setinin hem **Part A** hem de **Part B** bölümleri için tanımlı olan `train`, `val` ve `test` listeleri arasında herhangi bir kesişim olup olmadığı incelenmiştir.

Yapılan kontroller sonucunda:
- Eğitim, doğrulama ve test kümeleri arasında **herhangi bir örtüşme olmadığı**,
- Her bir görüntünün **yalnızca tek bir kümeye ait olduğu**,
- Üç kümenin birleşiminin, ilgili bölümdeki tüm görüntüleri kapsadığı

doğrulanmıştır.

Bu doğrulama, deneysel sonuçların istatistiksel olarak geçerli olmasını ve test aşamasında elde edilen performansın ezberlemeye değil, gerçek genelleme yeteneğine dayandığını garanti etmektedir.
*

----------
----------
----------
----------

## Adım 2 — YOLO veri setini oluşturma (Train + Val + Test) — `build_scut_yolo.py`

Bu adımda amaç, SCUT-HEAD veri setinin **Part A** ve **Part B** bölümlerini kullanarak YOLO formatında **train**, **validation (val)** ve **test** klasörlerini **tek bir akışta** oluşturmaktır.  
Bu süreçte hem **görüntüler** hem de **etiketler (labels)** split listelerine göre doğru klasörlere yerleştirilir. İşlemin tek seferde yapılması, dosya eşleşme hatalarını (image–label mismatch) ve veri sızıntısı (data leakage) riskini azaltır.

### 2.1 Hedef klasör yapısını oluştur
Çıktı dizininde aşağıdaki klasörleri oluştur:

- `scut_head_yolo/images/train/`
- `scut_head_yolo/images/val/`
- `scut_head_yolo/images/test/`
- `scut_head_yolo/labels/train/`
- `scut_head_yolo/labels/val/`
- `scut_head_yolo/labels/test/`

> Not: Ham dataset klasörlerine (SCUT_HEAD_Part_A / SCUT_HEAD_Part_B) **dokunmuyoruz**; bu klasörler **raw** olarak kalır.

### 2.2 Split listelerini kaynak al
Aşağıdaki split dosyaları **ground-truth** (referans bölünme) kabul edilir:

**Part A**
- `ImageSets/Main/train.txt`
- `ImageSets/Main/val.txt`
- `ImageSets/Main/test.txt`

**Part B**
- `ImageSets/Main/train.txt`
- `ImageSets/Main/val.txt`
- `ImageSets/Main/test.txt`

Bu dosyalardaki her satır, ilgili split’e girecek görselin **ID** bilgisidir (ör. `PartB_00042`).

### 2.3 Her split için uygulanacak işlem (train / val / test)
Her split için (train, val, test) Part A ve Part B listeleri ayrı ayrı okunur ve aynı mantık uygulanır:

1. **Görsel dosyasını bul**
   - `JPEGImages/<ID>.jpg` (veya `.png/.jpeg`)

2. **XML anotasyonunu bul**
   - `Annotations/<ID>.xml`

3. **XML → YOLO label dönüşümü yap**
   - XML içindeki her `object` için bbox koordinatlarını al:
     - `(xmin, ymin, xmax, ymax)`
   - Görsel boyutunu XML’den oku:
     - `width`, `height`
   - YOLO formatına çevir (normalize):
     - `x_center = ((xmin + xmax) / 2) / width`
     - `y_center = ((ymin + ymax) / 2) / height`
     - `w = (xmax - xmin) / width`
     - `h = (ymax - ymin) / height`
   - Sınıf tek olduğu için `class_id = 0` kullanılır (projede **head** olarak yorumlanır).

4. **Çıktıları doğru split klasörüne yaz**
   - Görseli kopyala: `scut_head_yolo/images/<split>/<ID>.<ext>`
   - Label dosyasını yaz: `scut_head_yolo/labels/<split>/<ID>.txt`

### 2.4 Bu adımın sonunda yapılacak kontroller (zorunlu)
Veri seti üretildikten sonra aşağıdaki kontroller yapılacaktır:

- **Dosya sayıları**
  - `images/train/` beklenen: `A_train (1100) + B_train (1443) = 2543`
  - `images/val/` beklenen: `A_val (400) + B_val (462) = 862`
  - `images/test/` beklenen: `A_test (500) + B_test (500) = 1000`
  - `labels/<split>/` dosya sayıları, ilgili `images/<split>/` ile **uyumlu** olmalıdır (negatif örnekler için boş `.txt` dosyaları üretilebilir).

- **Leakage (kesişim) kontrolü**
  - `train ∩ val = ∅`
  - `train ∩ test = ∅`
  - `val ∩ test = ∅`

- **Görsel doğrulama**
  - Her split’ten rastgele birkaç görsel seçilerek bbox çizdirilir ve görsel–label eşleşmesi gözle kontrol edilir.

Bu kontroller geçmeden eğitim (fine-tuning) adımına geçilmeyecektir.


In [5]:
import shutil
import xml.etree.ElementTree as ET
from pathlib import Path
from PIL import Image

PART_A_ROOT = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Datasets\SCUT_HEAD_Part_A")
PART_B_ROOT = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Datasets\SCUT_HEAD_Part_B")
OUT_ROOT = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo")

IMG_DIR = "JPEGImages"
ANN_DIR = "Annotations"
SPLIT_DIR = Path("ImageSets") / "Main"

XML_LABEL_NAME = "person"
CLASS_ID = 0
IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def read_ids(txt_path: Path) -> list[str]:
    if not txt_path.exists():
        raise FileNotFoundError(f"Split dosyası bulunamadı: {txt_path}")
    ids = []
    for line in txt_path.read_text(encoding="utf-8", errors="ignore").splitlines():
        s = line.strip()
        if not s:
            continue
        s = s.replace(".jpg", "").replace(".jpeg", "").replace(".png", "")
        ids.append(s)
    return ids

def find_img_path(root: Path, image_id: str) -> Path:
    base = root / IMG_DIR
    for ext in IMG_EXTS:
        p = base / f"{image_id}{ext}"
        if p.exists():
            return p
    raise FileNotFoundError(f"Görsel bulunamadı: {base} | id={image_id}")

def find_xml_path(root: Path, image_id: str) -> Path:
    p = root / ANN_DIR / f"{image_id}.xml"
    if not p.exists():
        raise FileNotFoundError(f"XML bulunamadı: {p}")
    return p

def get_image_size(img_path: Path) -> tuple[int, int]:
    with Image.open(img_path) as im:
        w, h = im.size
    return int(w), int(h)

def safe_float(text: str, default: float = 0.0) -> float:
    try:
        return float(text)
    except Exception:
        return default

def voc_to_yolo_lines(xml_path: Path, img_path: Path) -> list[str]:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    w_img = h_img = 0.0
    size = root.find("size")
    if size is not None:
        w_img = safe_float((size.findtext("width") or "0"), 0.0)
        h_img = safe_float((size.findtext("height") or "0"), 0.0)

    if w_img <= 0 or h_img <= 0:
        w, h = get_image_size(img_path)
        w_img, h_img = float(w), float(h)

    lines = []
    for obj in root.findall("object"):
        name = (obj.findtext("name") or "").strip()
        if name != XML_LABEL_NAME:
            continue

        bb = obj.find("bndbox")
        if bb is None:
            continue

        xmin = safe_float(bb.findtext("xmin") or "0")
        ymin = safe_float(bb.findtext("ymin") or "0")
        xmax = safe_float(bb.findtext("xmax") or "0")
        ymax = safe_float(bb.findtext("ymax") or "0")

        if xmax < xmin:
            xmin, xmax = xmax, xmin
        if ymax < ymin:
            ymin, ymax = ymax, ymin

        xmin = max(0.0, min(xmin, w_img - 1))
        xmax = max(0.0, min(xmax, w_img - 1))
        ymin = max(0.0, min(ymin, h_img - 1))
        ymax = max(0.0, min(ymax, h_img - 1))

        bw = xmax - xmin
        bh = ymax - ymin
        if bw <= 0 or bh <= 0:
            continue

        cx = (xmin + xmax) / 2.0 / w_img
        cy = (ymin + ymax) / 2.0 / h_img
        bw_n = bw / w_img
        bh_n = bh / h_img

        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        bw_n = min(max(bw_n, 0.0), 1.0)
        bh_n = min(max(bh_n, 0.0), 1.0)

        lines.append(f"{CLASS_ID} {cx:.6f} {cy:.6f} {bw_n:.6f} {bh_n:.6f}")

    return lines

def build_split(split_name: str, ids_a: list[str], ids_b: list[str]):
    img_out = OUT_ROOT / "images" / split_name
    lbl_out = OUT_ROOT / "labels" / split_name
    ensure_dir(img_out)
    ensure_dir(lbl_out)

    missing = []
    written = 0

    def process_one(root: Path, image_id: str):
        nonlocal written
        img_src = find_img_path(root, image_id)
        xml_src = find_xml_path(root, image_id)

        img_dst = img_out / img_src.name
        shutil.copy2(img_src, img_dst)

        lbl_dst = lbl_out / f"{image_id}.txt"
        try:
            yolo_lines = voc_to_yolo_lines(xml_src, img_src)
            lbl_dst.write_text("\n".join(yolo_lines) + ("\n" if yolo_lines else ""), encoding="utf-8")
        except Exception as e:
            lbl_dst.write_text("", encoding="utf-8")
            missing.append((image_id, str(e)))
        written += 1

    for image_id in ids_a:
        process_one(PART_A_ROOT, image_id)
    for image_id in ids_b:
        process_one(PART_B_ROOT, image_id)

    img_count = len(list(img_out.glob("*.*")))
    lbl_count = len(list(lbl_out.glob("*.txt")))

    print(f"\n=== SPLIT: {split_name.upper()} ===")
    print(f"Beklenen ID sayısı: {len(ids_a) + len(ids_b)}")
    print(f"İşlenen ID sayısı: {written}")
    print(f"images/{split_name}: {img_count}")
    print(f"labels/{split_name}: {lbl_count}")
    print(f"Label üretim hatası: {len(missing)}")

    if missing:
        print("İlk 10 hata örneği:")
        for mid, msg in missing[:10]:
            print(f"- {mid}: {msg}")

def main():
    a_splits = PART_A_ROOT / SPLIT_DIR
    b_splits = PART_B_ROOT / SPLIT_DIR

    build_split("train", read_ids(a_splits / "train.txt"), read_ids(b_splits / "train.txt"))
    build_split("val", read_ids(a_splits / "val.txt"), read_ids(b_splits / "val.txt"))
    build_split("test", read_ids(a_splits / "test.txt"), read_ids(b_splits / "test.txt"))

    train_names = set(p.stem for p in (OUT_ROOT / "images" / "train").glob("*.*"))
    val_names = set(p.stem for p in (OUT_ROOT / "images" / "val").glob("*.*"))
    test_names = set(p.stem for p in (OUT_ROOT / "images" / "test").glob("*.*"))

    print("\n=== OUTPUT LEAK CHECK ===")
    print("train ∩ val :", len(train_names & val_names))
    print("train ∩ test:", len(train_names & test_names))
    print("val ∩ test  :", len(val_names & test_names))
    print("\nBitti. Çıktı klasörü:", OUT_ROOT)

if __name__ == "__main__":
    main()


=== SPLIT: TRAIN ===
Beklenen ID sayısı: 2543
İşlenen ID sayısı: 2543
images/train: 2543
labels/train: 2543
Label üretim hatası: 0

=== SPLIT: VAL ===
Beklenen ID sayısı: 862
İşlenen ID sayısı: 862
images/val: 862
labels/val: 862
Label üretim hatası: 0

=== SPLIT: TEST ===
Beklenen ID sayısı: 1000
İşlenen ID sayısı: 1000
images/test: 1000
labels/test: 1000
Label üretim hatası: 0

=== OUTPUT LEAK CHECK ===
train ∩ val : 0
train ∩ test: 0
val ∩ test  : 0

Bitti. Çıktı klasörü: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo


---
----
----
----


## Adım 3 — Split Oranlarını Yeniden Düzenleme (Train’i Güçlendirme) -- rebalance_splits.py

Bu adımın amacı, resmi (train/val/test) bölünmeler üzerinden oluşturduğumuz YOLO veri setinde **eğitim (train) örnek sayısını artırmak** ve aynı zamanda **doğrulama (val)** ile **test** kümelerini daha yönetilebilir boyutlara indirerek modelin öğrenmesini kolaylaştırmaktır.

Mevcut durumda test kümesi (1000 görüntü) toplam veri içinde yüksek bir paya sahiptir. Bu durum, değerlendirme güvenilirliğini artırsa da, **eğitim için kullanılabilecek veri miktarını azaltarak** model performansını sınırlayabilir. Bu nedenle, test ve validation kümelerindeki örnek sayısını hedeflediğimiz seviyelere çekip, geri kalan örnekleri train’e ekleyerek **modelin daha fazla örnek görmesini** ve **daha iyi öğrenmesini** hedefliyoruz.

Bu düzenleme yapılırken temel prensipler şunlardır:
- Train kümesi büyüt_toggle edilerek öğrenme kapasitesi artırılır.
- Val kümesi, eğitim sırasında model seçimi ve overfitting kontrolü için yeterli büyüklükte tutulur.
- Test kümesi, nihai performans raporlaması için makul bir boyutta tutulur.


#### Önemli: her taşıma image + label birlikte yapılacak.

In [6]:
import random
import shutil
from pathlib import Path

SRC = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo")
DST = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced")

TARGET_VAL = 350
TARGET_TEST = 700
SEED = 42

def ensure(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def copy_tree():
    for split in ["train", "val", "test"]:
        ensure(DST / "images" / split)
        ensure(DST / "labels" / split)

    for split in ["train", "val", "test"]:
        for p in (SRC / "images" / split).glob("*.*"):
            shutil.copy2(p, DST / "images" / split / p.name)
        for p in (SRC / "labels" / split).glob("*.txt"):
            shutil.copy2(p, DST / "labels" / split / p.name)

def list_stems(img_dir: Path):
    return sorted([p.stem for p in img_dir.glob("*.*") if p.is_file()])

def move_pair(stem: str, from_split: str, to_split: str):
    img_from = DST / "images" / from_split
    lbl_from = DST / "labels" / from_split
    img_to = DST / "images" / to_split
    lbl_to = DST / "labels" / to_split

    img_src = None
    for p in img_from.glob(stem + ".*"):
        img_src = p
        break
    if img_src is None:
        raise FileNotFoundError(f"Image yok: {from_split}/{stem}")

    lbl_src = lbl_from / f"{stem}.txt"
    if not lbl_src.exists():
        raise FileNotFoundError(f"Label yok: {from_split}/{stem}.txt")

    shutil.move(str(img_src), str(img_to / img_src.name))
    shutil.move(str(lbl_src), str(lbl_to / lbl_src.name))

def count_split(split: str):
    img_n = len(list((DST / "images" / split).glob("*.*")))
    lbl_n = len(list((DST / "labels" / split).glob("*.txt")))
    return img_n, lbl_n

def leak_check():
    train = set(list_stems(DST / "images" / "train"))
    val = set(list_stems(DST / "images" / "val"))
    test = set(list_stems(DST / "images" / "test"))
    return len(train & val), len(train & test), len(val & test)

def main():
    random.seed(SEED)
    if DST.exists():
        raise RuntimeError(f"DST zaten var, sil veya ad değiştir: {DST}")

    copy_tree()

    val_stems = list_stems(DST / "images" / "val")
    test_stems = list_stems(DST / "images" / "test")

    if len(val_stems) < TARGET_VAL:
        raise ValueError("Val zaten hedefin altında.")
    if len(test_stems) < TARGET_TEST:
        raise ValueError("Test zaten hedefin altında.")

    move_from_val = len(val_stems) - TARGET_VAL
    move_from_test = len(test_stems) - TARGET_TEST

    val_pick = random.sample(val_stems, move_from_val)
    test_pick = random.sample(test_stems, move_from_test)

    for stem in val_pick:
        move_pair(stem, "val", "train")
    for stem in test_pick:
        move_pair(stem, "test", "train")

    for split in ["train", "val", "test"]:
        img_n, lbl_n = count_split(split)
        print(f"{split}: images={img_n} labels={lbl_n}")

    a, b, c = leak_check()
    print("leak check train∩val:", a)
    print("leak check train∩test:", b)
    print("leak check val∩test:", c)
    print("Bitti:", DST)

if __name__ == "__main__":
    main()

train: images=3355 labels=3355
val: images=350 labels=350
test: images=700 labels=700
leak check train∩val: 0
leak check train∩test: 0
leak check val∩test: 0
Bitti: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced


Beklenen çıktı (yaklaşık):

* train: images=3355 labels=3355

* val: images=350 labels=350

* test: images=700 labels=700

* leak check hepsi 0

### Bu işlem tamamlandığında ilk çözümlenmiş veri setini projeden kaldırıyoruz. 
##### > **"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo"**

----
----
----
---


## Adım 4 — YOLO Veri Tanımlama Dosyası (`dataset.yaml`) Oluşturma

Bu adımın amacı, hazırladığımız YOLO formatındaki veri setini Ultralytics YOLO eğitim pipeline’ına tanıtmaktır.  
YOLO eğitim komutu, train/val/test görüntülerinin nerede bulunduğunu ve sınıf isimlerini `dataset.yaml` dosyasından okur.

### 4.1 `dataset.yaml` dosyasının konumu

`dataset.yaml` dosyası, veri setinin **kök dizinine** (yani `images/` ve `labels/` klasörleri ile aynı seviyeye) kaydedilmelidir:

- `C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\dataset.yaml`

Bu sayede YAML içindeki `train`, `val`, `test` yolları **göreli (relative)** yazılabilir ve taşınabilirlik artar.

### 4.2 `dataset.yaml` dosya içeriği

Aşağıdaki içerik, tek sınıflı (head) bir detection veri seti için yeterlidir:

```yaml
path: .
train: images/train
val: images/val
test: images/test

names:
  0: head
```
**Bu adım tamamlandıktan sonra eğitim aşamasında data=dataset.yaml parametresi kullanılarak model fine-tuning sürecine geçilecektir.**

----
-----